In [1]:
# Import necessary libraries;
import numpy as np
from numpy import mean
from numpy import std


import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set_style('white')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from mpl_toolkits.mplot3d import Axes3D
from numpy import genfromtxt

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.datasets import load_digits, make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn import tree

In [2]:
leagueData = pd.read_csv('LeagueMatches.csv')
del leagueData['Unnamed: 0']
del leagueData['matchId']
leagueData.head()

,blue_win,blueGold,blueMinionsKilled,blueJungleMinionsKilled,blueAvgLevel,redGold,redMinionsKilled,redJungleMinionsKilled,redAvgLevel,blueChampKills,blueHeraldKills,blueDragonKills,blueTowersDestroyed,redChampKills,redHeraldKills,redDragonKills,redTowersDestroyed
0,0,24575.0,349.0,89.0,8.6,25856.0,346.0,80.0,9.2,6.0,1.0,0.0,1.0,12.0,2.0,0.0,1.0
1,0,27210.0,290.0,36.0,9.0,28765.0,294.0,92.0,9.4,20.0,0.0,0.0,0.0,19.0,2.0,0.0,0.0
2,1,32048.0,346.0,92.0,9.4,25305.0,293.0,84.0,9.4,17.0,3.0,0.0,0.0,11.0,0.0,0.0,4.0
3,0,20261.0,223.0,60.0,8.2,30429.0,356.0,107.0,9.4,7.0,0.0,0.0,3.0,16.0,3.0,0.0,0.0
4,1,30217.0,376.0,110.0,9.8,23889.0,334.0,60.0,8.8,16.0,3.0,0.0,0.0,8.0,0.0,0.0,2.0


In [3]:
X = leagueData.iloc[:,1:17].values
y = leagueData.iloc[:,0].values
#To be used for validation purposes with best model for each trial 5000 train and 45000 test
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.90, random_state=42)

# SUPPORT VECTOR MACHINE TRIAL ONE ON LEAGUE OF LEGENDS DATASET

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = leagueData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,1:17].values
    ySet = random5000DataPoints.iloc[:,0].values
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro', 'f1_micro','precision_micro','roc_auc_ovo','jaccard','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1, )
    TRIAL_1_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL ONE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   35.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   45.2s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   47.8s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   53.6s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   57.6s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL ONE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  5.4min finished


In [6]:
TRIAL_1_SVM.cv_results_

{'mean_fit_time': array([3.86362329, 4.03316803, 3.88223829, 3.77594752, 3.85801792,
        3.90145521, 3.88484097, 4.00194182, 4.31220865, 4.80683389,
        4.32611995, 4.52329054, 4.69463739, 4.2718739 , 4.22663522,
        3.80307121, 3.43635488, 4.03757229, 4.13775911, 4.11854224,
        4.35714746, 4.76019382, 4.46884284, 3.1558136 , 2.50085077,
        2.70983062, 3.9092628 , 3.98462648, 3.94208851, 4.01124949,
        4.19240565, 3.56746769, 2.42278409, 2.32099576, 2.83793979,
        4.87058883, 4.02516122, 4.20031209, 4.00374293, 3.10597129,
        2.42898927, 2.26274471, 2.46111636, 3.75382833, 4.94575443,
        4.29629507, 4.46093593, 3.6026988 , 2.57281213, 2.27185302,
        2.60604081, 4.12714982, 3.88884454, 4.57573509, 4.12264504,
        3.41924057, 2.41627774, 2.26014338, 2.53337889, 4.04998298,
        4.3517426 , 3.78145189, 4.46483932, 3.24008694, 2.4451026 ,
        2.27906022, 2.67289858, 4.13905911, 4.3202157 , 4.46243777,
        4.03797264, 4.58133945,

In [7]:
print("---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 1,
 'classifier__gamma': 1e-05,
 'classifier__kernel': 'linear'}

In [8]:
print("---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_f1_micro']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'sigmoid'}

In [9]:
print("---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100000,
 'classifier__gamma': 1e-06,
 'classifier__kernel': 'rbf'}

In [10]:
print("---------------------------------TRIAL ONE JACCARD SCORE RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_jaccard']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_jaccard']) ]

---------------------------------TRIAL ONE JACCARD SCORE RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'sigmoid'}

In [12]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best JACCARD results to train on test set, LR_Trial 1
p = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_jaccard']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_jaccard = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best JACCARD hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_jaccard)))
print("------------------------------------------------------------------------------------------------")

------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best NEG LOG LOSS hyperparameters :0.7862787192253231
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best F1 hyperparameters :0.7857991138720138
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best ROC_AUC hyperparameters :0.7857534371716988
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best JACCARD hyperparameters :0.7857991138720138
------------------------------------------------------------------------------------------------


# SUPPORT VECTOR MACHINE TRIAL TWO ON LEAGUE OF LEGENDS DATASET

In [13]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = leagueData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,1:17].values
    ySet = random5000DataPoints.iloc[:,0].values
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro', 'f1_micro','precision_micro','roc_auc_ovo','jaccard','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_2_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL TWO RESULTS ---------------------------------------------")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   37.8s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   43.8s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   46.8s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   52.0s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   55.9s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL TWO RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  5.5min finished


In [14]:
TRIAL_2_SVM.cv_results_

{'mean_fit_time': array([3.79516435, 4.51588321, 4.68893161, 4.16548209, 4.12965202,
        4.51688471, 4.27317472, 4.73727398, 4.70354481, 4.63218427,
        4.25575981, 4.34113369, 4.39087548, 4.93354335, 4.67001591,
        3.8348978 , 3.62201471, 4.01695428, 4.07010074, 4.06549578,
        4.29889703, 4.31601262, 4.03236771, 3.39141588, 2.79370217,
        2.87797523, 4.13585653, 4.0035449 , 4.17198734, 3.88103762,
        3.98082385, 3.0890564 , 2.26204543, 2.10160723, 2.67690191,
        4.54831133, 3.85611629, 3.99023213, 4.00934806, 3.03821311,
        2.32790222, 2.14664536, 2.44139924, 3.60810266, 4.69113436,
        4.11914282, 4.36525474, 3.5333384 , 2.61204667, 2.38725276,
        2.45401077, 4.10162725, 3.82689104, 4.54080467, 4.04868202,
        3.30224009, 2.54488811, 2.31408982, 2.71253309, 4.26416683,
        4.55331607, 3.62141466, 4.41059308, 3.10647135, 2.437396  ,
        2.21310349, 2.8154212 , 4.08531322, 4.39457889, 4.33082428,
        3.90906205, 4.62017303,

In [15]:
print("---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100000,
 'classifier__gamma': 1e-06,
 'classifier__kernel': 'rbf'}

In [16]:
print("---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_f1_micro']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 1,
 'classifier__gamma': 0.1,
 'classifier__kernel': 'rbf'}

In [17]:
print("---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100000,
 'classifier__gamma': 1e-06,
 'classifier__kernel': 'rbf'}

In [18]:
print("---------------------------------TRIAL TWO JACCARD SCORE RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_jaccard']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_jaccard']) ]

---------------------------------TRIAL TWO JACCARD SCORE RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 0.001,
 'classifier__gamma': 0.1,
 'classifier__kernel': 'poly'}

In [19]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best JACCARD results to train on test set, LR_Trial 1
p = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_jaccard']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_jaccard = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best JACCARD hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_jaccard)))
print("------------------------------------------------------------------------------------------------")

------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best NEG LOG LOSS hyperparameters :0.7857534371716988
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best F1 hyperparameters :0.78271593660074
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best ROC_AUC hyperparameters :0.7857534371716988
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best JACCARD hyperparameters :0.7790618005755264
------------------------------------------------------------------------------------------------


# SUPPORT VECTOR MACHINE TRIAL THREE ON LEAGUE OF LEGENDS DATASET

In [20]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = leagueData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,1:17].values
    ySet = random5000DataPoints.iloc[:,0].values
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro', 'f1_micro','precision_micro','roc_auc_ovo','jaccard','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_3_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL THREE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   34.6s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   40.5s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   43.4s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   49.0s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   53.4s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL THREE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  5.4min finished


In [21]:
TRIAL_3_SVM.cv_results_

{'mean_fit_time': array([3.96080747, 3.93628521, 4.09962516, 3.83499842, 4.01965733,
        4.04858193, 3.95670204, 3.80737419, 3.75382824, 3.97481871,
        3.87853603, 4.03096676, 4.06769829, 3.93108029, 3.96510997,
        3.37400188, 3.27931972, 3.87052836, 3.93148103, 3.83930244,
        3.8914475 , 3.8848413 , 3.80857506, 3.08685474, 2.3260005 ,
        2.52797356, 4.08971729, 3.8787364 , 4.08020935, 3.9416894 ,
        4.42970958, 3.70418558, 2.37934642, 2.21800818, 2.85625672,
        5.03062687, 4.08791552, 4.42950969, 4.16768403, 3.29122987,
        2.40006437, 2.21570573, 2.5073565 , 3.58328176, 4.47624955,
        3.96791329, 3.95420036, 3.16872492, 2.34371562, 2.17436948,
        2.43149123, 3.81708255, 3.69557838, 4.37025833, 3.74602175,
        3.04982309, 2.27645817, 2.15955744, 2.55229559, 4.03557034,
        4.215025  , 3.59278979, 4.48425713, 3.39762254, 2.75316801,
        2.48853951, 2.55199428, 4.16458106, 4.2608644 , 4.12734938,
        3.5595614 , 4.47374735,

In [22]:
print("---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 1000,
 'classifier__gamma': 0.0001,
 'classifier__kernel': 'rbf'}

In [23]:
print("---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_f1_micro']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 0.1,
 'classifier__gamma': 0.1,
 'classifier__kernel': 'rbf'}

In [24]:
print("---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 1e-05,
 'classifier__kernel': 'rbf'}

In [25]:
print("---------------------------------TRIAL THREE JACCARD SCORE RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_jaccard']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_jaccard']) ]

---------------------------------TRIAL THREE JACCARD SCORE RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 0.01,
 'classifier__gamma': 0.1,
 'classifier__kernel': 'rbf'}

In [26]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best JACCARD results to train on test set, LR_Trial 1
p = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_jaccard']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_jaccard = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best JACCARD hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_jaccard)))
print("------------------------------------------------------------------------------------------------")

------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best NEG LOG LOSS hyperparameters :0.7860960124240625
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best F1 hyperparameters :0.7860274973735897
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best ROC_AUC hyperparameters :0.7860503357237473
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best JACCARD hyperparameters :0.7855022153199653
------------------------------------------------------------------------------------------------


# SUPPORT VECTOR MACHINE TRIAL FOUR ON LEAGUE OF LEGENDS DATASET

In [27]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = leagueData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,1:17].values
    ySet = random5000DataPoints.iloc[:,0].values
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro', 'f1_micro','precision_micro','roc_auc_ovo','jaccard','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_4_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FOUR RESULTS ---------------------------------------------")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   49.6s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   54.4s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FOUR RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  5.4min finished


In [28]:
TRIAL_4_SVM.cv_results_

{'mean_fit_time': array([3.87092934, 3.81698146, 3.95199766, 3.85661697, 4.07880836,
        4.16568236, 4.22353201, 3.82308812, 3.86972775, 4.10673156,
        3.82478857, 3.86712599, 3.89264765, 3.80837555, 3.88113828,
        3.30924544, 3.22867651, 3.82509007, 3.9301795 , 4.02576241,
        4.29589458, 4.60996523, 4.46964388, 3.39211721, 2.65768528,
        3.11878219, 4.10563045, 4.48575807, 4.38687205, 3.97171488,
        4.01745467, 3.16241946, 2.35392427, 2.10781169, 2.78349409,
        4.68452744, 4.03927412, 3.93358288, 4.09812431, 3.37740502,
        2.43319283, 2.35652552, 2.52827358, 3.76213555, 4.79672518,
        4.01685486, 4.24234939, 3.69137449, 2.34041338, 2.82192678,
        2.8133194 , 4.33652868, 4.56842895, 4.98608789, 4.28548579,
        3.49410543, 2.50585513, 2.43599491, 2.62585769, 4.08891635,
        4.42430491, 3.81287947, 4.54490838, 3.22217126, 2.3828496 ,
        2.34741912, 2.58011789, 4.00754619, 4.26046443, 4.09532189,
        3.61500888, 4.4498271 ,

In [29]:
print("---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'rbf'}

In [30]:
print("---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_f1_micro']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 1000,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'sigmoid'}

In [31]:
print("---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'rbf'}

In [32]:
print("---------------------------------TRIAL FOUR JACCARD SCORE RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_jaccard']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_jaccard']) ]

---------------------------------TRIAL FOUR JACCARD SCORE RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.0001,
 'classifier__kernel': 'rbf'}

In [33]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best JACCARD results to train on test set, LR_Trial 1
p = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_jaccard']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_jaccard = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best JACCARD hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_jaccard)))

------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best NEG LOG LOSS hyperparameters :0.7862102041748504
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best F1 hyperparameters :0.7829214817521583
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best ROC_AUC hyperparameters :0.7861645274745352
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best JACCARD hyperparameters :0.7777600146165441


# SUPPORT VECTOR MACHINE TRIAL FIVE ON LEAGUE OF LEGENDS DATASET

In [34]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = leagueData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,1:17].values
    ySet = random5000DataPoints.iloc[:,0].values
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro', 'f1_micro','precision_micro','roc_auc_ovo','jaccard','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_5_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FIVE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   20.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   41.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   49.9s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   54.3s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FIVE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  5.4min finished


In [35]:
TRIAL_5_SVM.cv_results_

{'mean_fit_time': array([3.72290201, 3.79035916, 3.86242113, 3.83299661, 4.06499619,
        4.04407806, 4.07640476, 4.39347849, 4.34974136, 4.57333283,
        4.04688072, 4.32381878, 4.53239784, 4.2099205 , 4.15977764,
        3.71149216, 3.31194844, 3.89114604, 3.96200743, 3.92567596,
        3.92527575, 4.13355489, 3.82849264, 3.08685479, 2.3293035 ,
        2.54138603, 3.79236202, 3.81848369, 3.79436336, 3.80337143,
        3.79516382, 3.15881667, 2.34641781, 2.27956038, 2.79900708,
        4.89350877, 3.93918734, 4.19550791, 4.57033005, 3.53904471,
        3.18513923, 2.59192915, 2.74245882, 3.91636744, 4.78591604,
        4.13185301, 4.06379495, 3.26330681, 2.44460254, 2.26284628,
        2.59222941, 3.97692003, 3.84830947, 4.51308203, 3.78395414,
        3.16722398, 2.28856754, 2.21080184, 2.50615525, 3.97992344,
        4.34103298, 3.69387689, 4.66581278, 3.25770187, 2.60283813,
        2.63276443, 2.64277248, 4.23143897, 4.52318959, 4.14156179,
        3.71549501, 4.45323014,

In [36]:
print("---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 1,
 'classifier__gamma': 0.01,
 'classifier__kernel': 'sigmoid'}

In [37]:
print("---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_f1_micro']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 0.1,
 'classifier__gamma': 0.1,
 'classifier__kernel': 'rbf'}

In [38]:
print("---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10,
 'classifier__gamma': 0.001,
 'classifier__kernel': 'rbf'}

In [39]:
print("---------------------------------TRIAL FIVE JACCARD SCORE RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_jaccard']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_jaccard']) ]

---------------------------------TRIAL FIVE JACCARD SCORE RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 0.1,
 'classifier__gamma': 0.1,
 'classifier__kernel': 'rbf'}

In [ ]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best JACCARD results to train on test set, LR_Trial 1
p = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_jaccard']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_jaccard = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best JACCARD hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_jaccard)))

------------------------------------------------------------------------------------------------
Accuracy of Trial 1 Logistic Regression using best NEG LOG LOSS hyperparameters :0.7864385876764263
------------------------------------------------------------------------------------------------
